# Regression model in Keras
## A - Build a baseline model

### Import libraries

In [1]:
import pandas as pd
import numpy as np

### Load data

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check dataset

In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Sanity check

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Number of independent variables

In [8]:
n_cols = predictors.shape[1]

Randomly split the data into a training and test sets by holding 30% of the data for testing

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)
print('Training set shapes: {} {}'.format(X_train.shape, y_train.shape))
print('Testing set shapes: {} {}'.format(X_test.shape, y_test.shape))

Training set shapes: (721, 8) (721,)
Testing set shapes: (309, 8) (309,)


### Build a neural network

Characteristics:

* One hidden layer of 10 nodes, and a ReLU activation function
* Use the **adam** optimizer and the **mean squared error**  as the loss function

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Define regression model

In [15]:
def baseline_regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Compile the model

In [17]:
model = baseline_regression_model()

### Train the model on the training data using 50 epochs

In [18]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 1s 2ms/step - loss: 25518.8458 - val_loss: 20272.5533
Epoch 2/50
504/504 [==============================] - 0s 429us/step - loss: 15539.8593 - val_loss: 11159.3869
Epoch 3/50
504/504 [==============================] - 0s 437us/step - loss: 7538.3936 - val_loss: 4524.1465
Epoch 4/50
504/504 [==============================] - 0s 476us/step - loss: 2701.4458 - val_loss: 1333.8238
Epoch 5/50
504/504 [==============================] - 0s 401us/step - loss: 898.3173 - val_loss: 607.0625
Epoch 6/50
504/504 [==============================] - 0s 898us/step - loss: 620.7228 - val_loss: 535.0875
Epoch 7/50
504/504 [==============================] - 0s 478us/step - loss: 556.5969 - val_loss: 476.2024
Epoch 8/50
504/504 [==============================] - 0s 434us/step - loss: 488.9016 - val_loss: 431.3691
Epoch 9/50
504/504 [==============================] - 0s 470us/step - loss: 437.0172 - val_loss:

### Evaluate the model on the test data
Compute the mean squared error between the predicted concrete strength and the actual concrete strength

In [19]:
from sklearn.metrics import mean_squared_error
yhat = model.predict(X_test)
print("MSE: %.4f" % mean_squared_error(y_test , yhat))

MSE: 154.7548


### Repeat splitting, training and evaluation 50 times

Create a list of 50 MSE

In [22]:
mse_list = list()

for i in range(50):
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)
    # Compile and train the model
    model = baseline_regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    # Predict based on test set and calculate MSE
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test , yhat)
    mse_list.append(mse)
    print("Iteration #%d --- MSE: %.4f" % (i+1, mse))

Iteration #1 --- MSE: 406.4182
Iteration #2 --- MSE: 166.9024
Iteration #3 --- MSE: 174.2275
Iteration #4 --- MSE: 398.0698
Iteration #5 --- MSE: 173.6569
Iteration #6 --- MSE: 258.2557
Iteration #7 --- MSE: 234.2780
Iteration #8 --- MSE: 198.8055
Iteration #9 --- MSE: 342.0200
Iteration #10 --- MSE: 142.3787
Iteration #11 --- MSE: 1025.3545
Iteration #12 --- MSE: 122.8591
Iteration #13 --- MSE: 373.3029
Iteration #14 --- MSE: 249.4491
Iteration #15 --- MSE: 1169.2828
Iteration #16 --- MSE: 288.4709
Iteration #17 --- MSE: 220.5240
Iteration #18 --- MSE: 305.2515
Iteration #19 --- MSE: 500.5383
Iteration #20 --- MSE: 134.4287
Iteration #21 --- MSE: 434.6075
Iteration #22 --- MSE: 717.6661
Iteration #23 --- MSE: 159.5513
Iteration #24 --- MSE: 1130.1056
Iteration #25 --- MSE: 273.3882
Iteration #26 --- MSE: 239.1952
Iteration #27 --- MSE: 154.7131
Iteration #28 --- MSE: 1256.1881
Iteration #29 --- MSE: 140.2253
Iteration #30 --- MSE: 120.4231
Iteration #31 --- MSE: 225.4212
Iteration #32

### Report the mean and the standard deviation of the mean squared errors

In [24]:
print("Mean of MSE: %.4f" % np.mean(mse_list))
print("Standard deviation of MSE: %.4f" % np.std(mse_list))

Mean of MSE: 334.7935
Standard deviation of MSE: 296.8284
